In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
seed = 2
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [2]:
train = pd.read_csv('./data/new_train.csv', na_values=-1)
test  = pd.read_csv('./data/new_test.csv', na_values=-1)

In [ ]:
sample_size = int(1e4)
sample = train[train.target == 1].sample(n=sample_size/2, random_state=seed)
sample = sample.append( train[train.target == 0].sample(n=sample_size/2, random_state=seed))
sample = sample.sample(frac=1, random_state=seed).reset_index(drop=True)
#sample.to_csv('./data/train_sample.csv', index=False)

In [3]:
target = train.target
train.drop(['target','id'], inplace=True, axis=1)
train.drop([ col for col in train.columns if col.startswith('ps_cont') ],axis=1, inplace=True)

In [4]:
ids = test.id
test.drop(['id'], inplace=True, axis=1)
test.drop([ col for col in test.columns if col.startswith('ps_cont') ],axis=1, inplace=True)

In [5]:
encoder = OneHotEncoder(sparse=False)
enc_train = None
enc_test  = None

In [ ]:
train = train.apply(lambda d : pd.Categorical(d).codes)
test = test.apply(lambda d : pd.Categorical(d).codes)

In [70]:
for feature in train.columns :
    encoded = encoder.fit_transform(train[feature].values.reshape(-1,1))
    if enc_train is None :
        enc_train = encoded
    else :
        enc_train = np.concatenate((enc_train, encoded), axis=1)

In [ ]:
for feature in test.columns :
    encoded = encoder.fit_transform(test[feature].values.reshape(-1,1))
    if enc_test is None :
        enc_test = encoded
    else :
        enc_test = np.concatenate((enc_test, encoded), axis=1)

In [94]:
x_train, x_test, y_train, y_test = train_test_split(enc_train, target, test_size=.2, random_state=seed)

In [98]:
model = XGBClassifier()

In [99]:
model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [100]:
pred = model.predict(x_test)

In [105]:
print classification_report(y_true=y_test, y_pred=pred)

             precision    recall  f1-score   support

          0       0.58      0.56      0.57      1043
          1       0.54      0.55      0.54       957

avg / total       0.56      0.56      0.56      2000

